# 🚀 Face Recognition Ensemble Training - Kaggle GPU Optimized

## 🎯 Objective
Train an ensemble of face recognition models using preprocessed VGGFace2 datasets with GPU acceleration.

### 📊 Datasets Used:
- **Training**: VGGFace2 Train 112x112 (Begin to 6000 identities)
- **Testing**: VGGFace2 Test 112x112
- **Preprocessing**: Already done (112x112, face-cropped)

### 🤖 Models:
- SE-ResNet-50 (Primary backbone)
- MobileFaceNet (Efficient model)
- Custom PyTorch ResNet (Baseline)

### ⚡ Optimizations:
- GPU acceleration with CUDA
- Mixed precision training
- Efficient data loading
- Advanced loss functions (ArcFace, CosFace)

## 1. 🔧 Environment Setup & GPU Configuration

In [1]:
# Essential imports and GPU setup
import os
import sys
import time
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm
import gc

# Deep learning imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models

# Image processing
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Metrics and utilities
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import kagglehub

warnings.filterwarnings('ignore')

# 🎮 GPU Configuration
print("🎮 GPU Configuration:")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    # Enable mixed precision for faster training
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
else:
    device = torch.device('cpu')
    print("⚠️ GPU not available, using CPU")

print(f"Device: {device}")

# Memory management
def cleanup_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

cleanup_memory()

🎮 GPU Configuration:
PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4
GPU Memory: 14.7 GB
Device: cuda


## 2. 📥 Dataset Loading & Preprocessing

In [2]:
# 📥 Download datasets
print("📥 Downloading VGGFace2 datasets...")

# Training dataset (Begin to 6000 identities)
print("\n🔄 Downloading training dataset...")
train_path = kagglehub.dataset_download("blackphantom55442664/vggface2-train112x112-beginto6000")
print(f"✅ Training dataset path: {train_path}")

# Test dataset
print("\n🔄 Downloading test dataset...")
test_path = kagglehub.dataset_download("hannenoname/vggface2-test-112x112")
print(f"✅ Test dataset path: {test_path}")

# Function to find actual data directory with images
def find_data_directory(root_path):
    """Find the directory that actually contains identity folders with images"""
    root_path = Path(root_path)

    def count_identity_dirs_with_images(directory):
        """Count directories that contain image files"""
        count = 0
        for item in directory.iterdir():
            if item.is_dir():
                # Check if this directory contains images
                image_files = [f for f in item.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
                if image_files:
                    count += 1
        return count

    # Check current directory first
    current_count = count_identity_dirs_with_images(root_path)
    best_path = root_path
    best_count = current_count

    print(f"   Root directory has {current_count} identity directories with images")

    # Check subdirectories for better structure
    for subdir in root_path.iterdir():
        if subdir.is_dir():
            subdir_count = count_identity_dirs_with_images(subdir)
            print(f"   Subdirectory '{subdir.name}' has {subdir_count} identity directories with images")

            if subdir_count > best_count:
                best_path = subdir
                best_count = subdir_count
                print(f"   ✅ Found better structure in: {subdir.name}")

    return best_path, best_count

# Explore dataset structure
print("\n📊 Dataset Structure Analysis:")

print(f"\n📁 Training dataset:")
train_root, train_count = find_data_directory(train_path)
print(f"   Using path: {train_root}")
print(f"   Found {train_count} identity directories")

if train_count > 0:
    # Show sample of first few directories
    train_subdirs = sorted([d for d in train_root.iterdir() if d.is_dir()])
    sample_counts = []
    for subdir in train_subdirs[:5]:
        image_count = len([f for f in subdir.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])
        if image_count > 0:
            sample_counts.append(image_count)
            print(f"   {subdir.name}: {image_count} images")

    total_train_images = sum(len([f for f in subdir.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])
                            for subdir in train_subdirs)
    print(f"   📊 Total training images: {total_train_images}")
    print(f"   👥 Total identities: {len(train_subdirs)}")

print(f"\n📁 Test dataset:")
test_root, test_count = find_data_directory(test_path)
print(f"   Using path: {test_root}")
print(f"   Found {test_count} identity directories")

if test_count > 0:
    test_subdirs = sorted([d for d in test_root.iterdir() if d.is_dir()])
    total_test_images = sum(len([f for f in subdir.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])
                           for subdir in test_subdirs)
    print(f"   📊 Total test images: {total_test_images}")
    print(f"   👥 Total identities: {len(test_subdirs)}")
else:
    print("   ⚠️ No valid test dataset found - will use train/validation split")

# Update paths to the actual data directories
train_path = str(train_root)
test_path = str(test_root)

📥 Downloading VGGFace2 datasets...

🔄 Downloading training dataset...


100%|██████████| 4.84G/4.84G [00:55<00:00, 94.2MB/s]

Extracting files...


✅ Training dataset path: /root/.cache/kagglehub/datasets/blackphantom55442664/vggface2-train112x112-beginto6000/versions/1

🔄 Downloading test dataset...


100%|██████████| 602M/602M [00:05<00:00, 118MB/s]

Extracting files...


Streaming output truncated to the last 5000 lines.
   Subdirectory 'n001253' has 0 identity directories with images
   Subdirectory 'n000402' has 0 identity directories with images
   Subdirectory 'n004622' has 0 identity directories with images
   Subdirectory 'n002449' has 0 identity directories with images
   Subdirectory 'n002394' has 0 identity directories with images
   Subdirectory 'n002428' has 0 identity directories with images
   Subdirectory 'n002364' has 0 identity directories with images
   Subdirectory 'n003317' has 0 identity directories with images
   Subdirectory 'n003023' has 0 identity directories with images
   Subdirectory 'n000720' has 0 identity directories with images
   Subdirectory 'n003832' has 0 identity directories with images
   Subdirectory 'n005251' has 0 identity directories with images
   Subdirectory 'n003813' has 0 identity directories with images
   Subdirectory 'n001308' has 0 identity directories with images
   Subdirectory 'n005519' has 0 identit

In [3]:
# 🎯 Custom Dataset Class for VGGFace2
class VGGFace2Dataset(Dataset):
    def __init__(self, root_dir, transform=None, max_samples_per_identity=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.max_samples_per_identity = max_samples_per_identity

        # Build dataset index
        self.samples = []
        self.labels = []
        self.identity_names = []

        print(f"📂 Building dataset index from {root_dir}...")

        # Get identity directories directly from the provided path
        identity_dirs = sorted([d for d in self.root_dir.iterdir() if d.is_dir()])

        if not identity_dirs:
            raise ValueError(f"No identity directories found in {root_dir}")

        # Filter out empty directories and count images
        valid_identity_dirs = []
        total_images_found = 0

        for identity_dir in identity_dirs:
            image_files = [f for f in identity_dir.iterdir()
                          if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
            if image_files:
                valid_identity_dirs.append(identity_dir)
                total_images_found += len(image_files)

        identity_dirs = valid_identity_dirs
        print(f"📊 Found {len(identity_dirs)} valid identity directories with {total_images_found} total images")

        if not identity_dirs:
            raise ValueError(f"No identity directories with images found in {root_dir}")

        for label_idx, identity_dir in enumerate(tqdm(identity_dirs, desc="Processing identities")):
            self.identity_names.append(identity_dir.name)

            # Get all image files
            image_files = [f for f in identity_dir.iterdir()
                          if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]

            # Limit samples per identity if specified
            if max_samples_per_identity and len(image_files) > max_samples_per_identity:
                image_files = image_files[:max_samples_per_identity]

            for image_file in image_files:
                self.samples.append(str(image_file))
                self.labels.append(label_idx)

        print(f"✅ Dataset built: {len(self.samples)} images, {len(self.identity_names)} identities")

        if len(self.samples) == 0:
            raise ValueError(f"No images found in dataset: {root_dir}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path = self.samples[idx]
        label = self.labels[idx]

        # Load image
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            # Fallback to PIL
            try:
                image = Image.open(image_path).convert('RGB')
                image = np.array(image)
            except Exception as e2:
                print(f"⚠️ Failed to load image {image_path}: {e2}")
                # Return a black image as fallback
                image = np.zeros((112, 112, 3), dtype=np.uint8)

        # Apply transforms
        if self.transform:
            if isinstance(self.transform, A.Compose):
                transformed = self.transform(image=image)
                image = transformed['image']
            else:
                image = self.transform(image)

        return image, label

# 🎨 Define transforms for training and validation
def get_transforms(image_size=112, is_training=True):
    if is_training:
        return A.Compose([
            A.Resize(image_size, image_size),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.3),
            A.CoarseDropout(max_holes=1, max_height=16, max_width=16, p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(image_size, image_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])

# ✅ Create datasets with smart test dataset detection
print("\n🎯 Creating datasets...")

# Create training dataset
try:
    train_dataset = VGGFace2Dataset(
        root_dir=train_path,
        transform=get_transforms(is_training=True),
        max_samples_per_identity=20  # Limit for faster training - remove for full dataset
    )
    print(f"✅ Training dataset created successfully!")
    print(f"   Training: {len(train_dataset)} images, {len(train_dataset.identity_names)} identities")
except Exception as e:
    print(f"❌ Error creating training dataset: {e}")
    raise

# Smart test dataset creation with automatic nested structure detection
test_dataset = None
use_train_val_split = False

print(f"\n🔍 Analyzing test dataset structure...")
test_root = Path(test_path)
best_test_path = None
max_images = 0

# Search for the best test data directory
for item in test_root.rglob("*"):
    if item.is_dir():
        # Check if this directory contains images
        image_files = [f for f in item.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
        if len(image_files) > 0:
            # Check if parent directory has multiple identity directories
            parent_dirs = [d for d in item.parent.iterdir() if d.is_dir()]
            identity_dirs_with_images = 0
            total_images_in_parent = 0

            for d in parent_dirs:
                try:
                    dir_images = [f for f in d.iterdir() if f.suffix.lower() in ['.jpg', '.jpeg', '.png']]
                    if dir_images:
                        identity_dirs_with_images += 1
                        total_images_in_parent += len(dir_images)
                except:
                    continue

            # If we found a directory with many identity folders, it's likely the main dataset
            if identity_dirs_with_images >= 10 and total_images_in_parent > max_images:
                max_images = total_images_in_parent
                best_test_path = item.parent
                print(f"   Found test data: {best_test_path.name} ({identity_dirs_with_images} identities, {total_images_in_parent} images)")

# Try to create test dataset with detected path
if best_test_path and max_images > 0:
    try:
        print(f"\n🎯 Creating test dataset from: {best_test_path}")
        test_dataset = VGGFace2Dataset(
            root_dir=best_test_path,
            transform=get_transforms(is_training=False),
            max_samples_per_identity=10  # Limit for faster evaluation
        )
        print(f"✅ Test dataset created successfully!")
        print(f"   Testing: {len(test_dataset)} images, {len(test_dataset.identity_names)} identities")
    except Exception as e:
        print(f"⚠️ Failed to create test dataset: {e}")
        use_train_val_split = True
else:
    print(f"⚠️ No valid test data structure found")
    use_train_val_split = True

# Fallback: Create train/validation split if test dataset failed
if use_train_val_split:
    print(f"\n🔄 Using training dataset split for validation...")
    from torch.utils.data import random_split

    # Split the training dataset
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size

    train_subset, test_dataset = random_split(
        train_dataset,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)  # For reproducible splits
    )

    # Update train_dataset to use the subset
    train_dataset = train_subset

    print(f"✅ Created train/validation split:")
    print(f"   Training: {len(train_dataset)} images")
    print(f"   Validation: {len(test_dataset)} images")

print(f"\n📊 Final Dataset Summary:")
print(f"   Training: {len(train_dataset)} images")
print(f"   Testing/Validation: {len(test_dataset)} images")
print(f"   Split type: {'Train/Val Split' if use_train_val_split else 'Separate Test Set'}")

# Verify both datasets have samples
if len(train_dataset) == 0:
    raise ValueError("Training dataset is empty!")
if len(test_dataset) == 0:
    raise ValueError("Test/validation dataset is empty!")

print(f"✅ Dataset creation complete! Ready for training 🚀")


🎯 Creating datasets...
📂 Building dataset index from /root/.cache/kagglehub/datasets/blackphantom55442664/vggface2-train112x112-beginto6000/versions/1...
📊 Found 5547 valid identity directories with 1829703 total images


Processing identities:   0%|          | 0/5547 [00:00<?, ?it/s]

✅ Dataset built: 110940 images, 5547 identities
✅ Training dataset created successfully!
   Training: 110940 images, 5547 identities

🔍 Analyzing test dataset structure...
   Found test data: test_processed (500 identities, 152618 images)

🎯 Creating test dataset from: /root/.cache/kagglehub/datasets/hannenoname/vggface2-test-112x112/versions/1/test_processed/test_processed
📂 Building dataset index from /root/.cache/kagglehub/datasets/hannenoname/vggface2-test-112x112/versions/1/test_processed/test_processed...
📊 Found 500 valid identity directories with 152618 total images


Processing identities:   0%|          | 0/500 [00:00<?, ?it/s]

✅ Dataset built: 5000 images, 500 identities
✅ Test dataset created successfully!
   Testing: 5000 images, 500 identities

📊 Final Dataset Summary:
   Training: 110940 images
   Testing/Validation: 5000 images
   Split type: Separate Test Set
✅ Dataset creation complete! Ready for training 🚀


## 3. 🏗️ Model Architecture Definitions

In [ ]:
# 🏗️ GPU-OPTIMIZED Face Recognition Models

# 1. SE-ResNet-50 with ArcFace Head - OPTIMIZED
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SEResNet50(nn.Module):
    def __init__(self, num_classes, embedding_dim=512):
        super().__init__()
        # Use pretrained ResNet50 as backbone
        resnet = models.resnet50(pretrained=True)

        # Remove final layers
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])

        # Add SE blocks
        self.se_block = SEBlock(2048)

        # Feature extraction head - OPTIMIZED
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(0.3)  # Reduced dropout for faster training
        self.embedding = nn.Linear(2048, embedding_dim)
        # Optimized BatchNorm for mixed precision
        self.bn = nn.BatchNorm1d(embedding_dim, eps=1e-5, momentum=0.1)

        # Classification head
        self.classifier = nn.Linear(embedding_dim, num_classes)
        
        # Initialize weights for faster convergence
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x, return_embedding=False):
        # Backbone features
        features = self.backbone(x)
        features = self.se_block(features)

        # Global pooling and embedding
        pooled = self.global_pool(features).flatten(1)
        pooled = self.dropout(pooled)
        embedding = self.embedding(pooled)
        
        # BatchNorm with mixed precision support
        embedding = self.bn(embedding)

        if return_embedding:
            return F.normalize(embedding, p=2, dim=1)

        # Classification
        logits = self.classifier(embedding)
        return logits, F.normalize(embedding, p=2, dim=1)

# 2. MobileFaceNet - OPTIMIZED
class MobileFaceNet(nn.Module):
    def __init__(self, num_classes, embedding_dim=512):
        super().__init__()
        # Use MobileNetV2 as backbone
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.backbone = mobilenet.features

        # Feature extraction head - OPTIMIZED
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(0.1)  # Lower dropout for mobile model
        self.embedding = nn.Linear(1280, embedding_dim)
        # Optimized BatchNorm for mixed precision
        self.bn = nn.BatchNorm1d(embedding_dim, eps=1e-5, momentum=0.1)

        # Classification head
        self.classifier = nn.Linear(embedding_dim, num_classes)
        
        # Initialize weights for faster convergence
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x, return_embedding=False):
        # Backbone features
        features = self.backbone(x)

        # Global pooling and embedding
        pooled = self.global_pool(features).flatten(1)
        pooled = self.dropout(pooled)
        embedding = self.embedding(pooled)
        
        # BatchNorm with mixed precision support
        embedding = self.bn(embedding)

        if return_embedding:
            return F.normalize(embedding, p=2, dim=1)

        # Classification
        logits = self.classifier(embedding)
        return logits, F.normalize(embedding, p=2, dim=1)

# 3. OPTIMIZED ArcFace Loss Function
class ArcFaceLoss(nn.Module):
    def __init__(self, embedding_dim, num_classes, margin=0.5, scale=64.0):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_classes = num_classes
        self.margin = margin
        self.scale = scale

        # Weight matrix with proper initialization
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, embedding_dim))
        nn.init.xavier_uniform_(self.weight)
        
        # Register buffer for cos and sin values to avoid recomputation
        self.register_buffer('cos_m', torch.cos(torch.tensor(margin)))
        self.register_buffer('sin_m', torch.sin(torch.tensor(margin)))
        self.register_buffer('threshold', torch.cos(torch.tensor(3.14159 - margin)))
        self.register_buffer('mm', torch.sin(torch.tensor(3.14159 - margin)) * margin)

    def forward(self, embeddings, labels):
        # Normalize weights and embeddings for numerical stability
        normalized_weights = F.normalize(self.weight, p=2, dim=1)
        normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

        # Compute cosine similarity
        cosine = F.linear(normalized_embeddings, normalized_weights)
        cosine = torch.clamp(cosine, -1.0 + 1e-7, 1.0 - 1e-7)

        # Compute sine from cosine
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))

        # Apply margin - optimized computation
        phi = cosine * self.cos_m - sine * self.sin_m

        # Handle numerical stability
        phi = torch.where(cosine > self.threshold, phi, cosine - self.mm)

        # Create one-hot labels efficiently
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1, 1), 1)

        # Apply margin only to target class
        logits = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        logits = logits * self.scale

        return logits

print("✅ GPU-OPTIMIZED model architectures defined")
print("🚀 Optimizations applied:")
print("   - Faster weight initialization")
print("   - Mixed precision compatible BatchNorm")
print("   - Optimized ArcFace with precomputed trigonometric values")
print("   - Reduced dropout for faster convergence")
print("   - Numerical stability improvements")
print("   - Memory efficient computations")

✅ Model architectures defined
   🏗️ SE-ResNet-50 with SE blocks
   📱 MobileFaceNet (efficient)
   🎯 ArcFace loss function


## 4. 🎯 Training Pipeline Setup

In [ ]:
# 🎯 HIGH-PERFORMANCE Training Configuration - Optimized for Speed & GPU Utilization
class TrainingConfig:
    # Model parameters
    embedding_dim = 512

    # Training parameters - HEAVILY OPTIMIZED FOR SPEED
    batch_size = 96 if torch.cuda.is_available() else 16  # INCREASED from 32 to 96 for max GPU usage
    num_epochs = 12  # REDUCED from 20 to 12 for faster training
    learning_rate = 3e-3  # INCREASED from 1e-3 to 3e-3 for faster convergence
    weight_decay = 5e-5  # REDUCED weight decay for faster training

    # ArcFace parameters
    arcface_margin = 0.4  # REDUCED margin for easier learning
    arcface_scale = 32.0  # REDUCED scale for more stable gradients

    # Scheduler parameters - More aggressive
    step_size = 4  # REDUCED from 7 to 4 for frequent LR updates
    gamma = 0.3  # INCREASED decay for aggressive scheduling

    # Mixed precision - ENABLED for speed
    use_amp = torch.cuda.is_available()

    # Checkpointing
    save_every = 2  # REDUCED from 5 to 2 for more frequent saves

    # PERFORMANCE optimizations
    num_workers = 6  # INCREASED workers for faster data loading
    pin_memory = True
    persistent_workers = True
    prefetch_factor = 6  # INCREASED prefetch for max throughput

config = TrainingConfig()

# Verify datasets exist before creating data loaders
if 'train_dataset' not in locals() or 'test_dataset' not in locals():
    raise ValueError("Datasets not created! Please run the dataset creation cell first.")

if len(train_dataset) == 0 or len(test_dataset) == 0:
    raise ValueError("One or both datasets are empty!")

# Create MAXIMUM PERFORMANCE data loaders
print("🚀 Creating HIGH-PERFORMANCE data loaders...")

try:
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        pin_memory=config.pin_memory,
        drop_last=True,
        persistent_workers=config.persistent_workers,
        prefetch_factor=config.prefetch_factor
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        pin_memory=config.pin_memory,
        drop_last=False,
        persistent_workers=config.persistent_workers,
        prefetch_factor=config.prefetch_factor
    )
    
    print("✅ High-performance data loaders created successfully!")
    
except Exception as e:
    print(f"⚠️ High-performance config failed: {e}")
    print("   Falling back to optimized-safe configuration...")
    
    # Fallback to more conservative but still optimized settings
    config.batch_size = 64
    config.num_workers = 4
    config.prefetch_factor = 4
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        pin_memory=config.pin_memory,
        drop_last=True,
        persistent_workers=config.persistent_workers,
        prefetch_factor=config.prefetch_factor
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        pin_memory=config.pin_memory,
        drop_last=False,
        persistent_workers=config.persistent_workers,
        prefetch_factor=config.prefetch_factor
    )
    
    print("✅ Optimized-safe configuration applied")

print(f"\n📊 HIGH-PERFORMANCE Training Configuration:")
print(f"   🔥 Batch size: {config.batch_size} (3X INCREASED for maximum GPU utilization)")
print(f"   ⚡ Epochs: {config.num_epochs} (40% REDUCED for efficiency)")
print(f"   🚀 Learning rate: {config.learning_rate} (3X INCREASED for rapid convergence)")
print(f"   🎯 Mixed precision: {config.use_amp} (ENABLED for 2X speed boost)")
print(f"   👥 Workers: {config.num_workers} (3X INCREASED for data loading)")
print(f"   📦 Prefetch: {config.prefetch_factor} batches ahead")
print(f"   🔄 Train batches: {len(train_loader)} (DRAMATICALLY REDUCED)")
print(f"   📊 Test batches: {len(test_loader)}")

# Calculate massive time savings
original_batches_per_epoch = len(train_dataset) // 32  # Original batch size
new_batches_per_epoch = len(train_loader)
batch_time_savings = (original_batches_per_epoch - new_batches_per_epoch) / original_batches_per_epoch * 100

original_total_batches = original_batches_per_epoch * 20  # Original epochs
new_total_batches = new_batches_per_epoch * config.num_epochs
total_time_savings = (original_total_batches - new_total_batches) / original_total_batches * 100

print(f"\n🚀 MASSIVE PERFORMANCE IMPROVEMENTS:")
print(f"   📈 Per epoch: ~{batch_time_savings:.1f}% fewer iterations")
print(f"   ⏱️ Total training: ~{total_time_savings:.1f}% faster completion")
print(f"   🧠 Learning rate: 3X faster potential convergence")
print(f"   💾 Memory: Mixed precision saves 50% GPU memory")
print(f"   🔄 Data loading: {config.num_workers}X parallel + prefetching")
print(f"   🎯 Expected training time: ~{100-total_time_savings:.0f}% of original")

# Verify optimized data loaders work
try:
    # Test train loader
    train_sample = next(iter(train_loader))
    print(f"\n   ✅ OPTIMIZED Train batch: {train_sample[0].shape}, labels: {train_sample[1].shape}")

    # Test validation loader  
    test_sample = next(iter(test_loader))
    print(f"   ✅ OPTIMIZED Test batch: {test_sample[0].shape}, labels: {test_sample[1].shape}")

    # Check GPU memory usage
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        cached = torch.cuda.memory_reserved() / 1024**3
        print(f"   🎮 GPU Memory: {allocated:.2f}GB allocated, {cached:.2f}GB cached")

    print(f"\n   🎉 HIGH-PERFORMANCE data loaders ready for MAXIMUM SPEED training!")

except Exception as e:
    print(f"   ❌ Error testing optimized data loaders: {e}")
    raise

📊 Training Configuration:
   Batch size: 32
   Epochs: 20
   Learning rate: 0.001
   Mixed precision: True
   Train batches: 3466
   Test batches: 157
   ✅ Train batch shape: torch.Size([32, 3, 112, 112]), labels: torch.Size([32])
   ✅ Test batch shape: torch.Size([32, 3, 112, 112]), labels: torch.Size([32])
   🚀 Data loaders ready for training!


In [ ]:
# 🚀 HIGH-SPEED Training Function - Optimized for Maximum Performance
def train_model(model, model_name, num_classes):
    print(f"\n🚀 HIGH-SPEED Training {model_name} with MAXIMUM GPU utilization...")

    # Move model to device and enable optimizations
    model = model.to(device)
    
    # Enable performance optimizations
    if hasattr(torch.backends.cudnn, 'benchmark'):
        torch.backends.cudnn.benchmark = True  # Optimize for consistent input sizes
    if hasattr(torch.backends.cudnn, 'deterministic'):
        torch.backends.cudnn.deterministic = False  # Allow non-deterministic for speed
    
    # Enable model optimizations
    model = torch.compile(model) if hasattr(torch, 'compile') else model  # PyTorch 2.0+ optimization

    # Loss functions with optimizations
    arcface_loss = ArcFaceLoss(
        embedding_dim=config.embedding_dim,
        num_classes=num_classes,
        margin=config.arcface_margin,
        scale=config.arcface_scale
    ).to(device)

    cross_entropy_loss = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing for better generalization

    # OPTIMIZED Optimizer - more aggressive settings
    optimizer = optim.AdamW(
        list(model.parameters()) + list(arcface_loss.parameters()),
        lr=config.learning_rate,
        weight_decay=config.weight_decay,
        betas=(0.9, 0.999),  # Optimized momentum parameters
        eps=1e-6  # Smaller epsilon for better numerical stability
    )

    # AGGRESSIVE Scheduler
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=config.learning_rate,
        epochs=config.num_epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.1,  # Warm up for 10% of training
        anneal_strategy='cos',  # Cosine annealing
        div_factor=10.0,  # Start with lr/10
        final_div_factor=100.0  # End with lr/100
    )

    # Mixed precision scaler with optimizations
    scaler = torch.cuda.amp.GradScaler(
        enabled=config.use_amp,
        init_scale=2**16,  # Higher initial scale
        growth_factor=2.0,
        backoff_factor=0.5,
        growth_interval=100  # More frequent scale updates
    ) if config.use_amp else None

    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_acc': [],
        'learning_rate': []
    }

    best_val_acc = 0.0
    
    # Early stopping for efficiency
    patience = 3
    no_improve_count = 0

    # Check if we have validation data
    has_validation = len(test_loader) > 0
    if not has_validation:
        print("⚠️ No validation data available - training without validation")

    print(f"🎯 Training with OPTIMIZED settings:")
    print(f"   📦 Batch size: {config.batch_size} (large batches for efficiency)")
    print(f"   🔄 Batches per epoch: {len(train_loader)}")
    print(f"   ⚡ Mixed precision: {config.use_amp}")
    print(f"   🎯 OneCycle LR: Max {config.learning_rate}")
    print(f"   🛡️ Early stopping: {patience} epochs patience")

    for epoch in range(config.num_epochs):
        epoch_start_time = time.time()
        
        # Training phase with optimizations
        model.train()
        arcface_loss.train()

        train_loss = 0.0
        train_correct = 0
        train_total = 0

        # Use faster progress bar
        pbar = tqdm(
            train_loader, 
            desc=f"🔥 Epoch {epoch+1}/{config.num_epochs}",
            leave=False,
            dynamic_ncols=True,
            ascii=True  # Faster rendering
        )

        for batch_idx, (images, labels) in enumerate(pbar):
            batch_start = time.time()
            
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)  # More efficient than zero_grad()

            if config.use_amp:
                with torch.cuda.amp.autocast():
                    logits, embeddings = model(images)
                    arcface_logits = arcface_loss(embeddings, labels)
                    loss = cross_entropy_loss(arcface_logits, labels)

                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)  # Unscale for gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
                scaler.step(optimizer)
                scaler.update()
            else:
                logits, embeddings = model(images)
                arcface_logits = arcface_loss(embeddings, labels)
                loss = cross_entropy_loss(arcface_logits, labels)

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
                optimizer.step()

            scheduler.step()  # OneCycle step every batch

            # Statistics
            train_loss += loss.item()
            _, predicted = torch.max(arcface_logits.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Update progress bar with detailed info
            current_acc = 100. * train_correct / train_total
            current_lr = optimizer.param_groups[0]['lr']
            batch_time = time.time() - batch_start
            
            pbar.set_postfix({
                'Loss': f'{loss.item():.3f}',
                'Acc': f'{current_acc:.1f}%',
                'LR': f'{current_lr:.2e}',
                'BT': f'{batch_time:.2f}s'
            })

            # Memory cleanup every 50 batches instead of 100
            if batch_idx % 50 == 0:
                cleanup_memory()

        # Validation phase (optimized)
        epoch_val_acc = 0.0
        if has_validation:
            model.eval()
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for images, labels in tqdm(test_loader, desc="🔍 Validation", leave=False, ascii=True):
                    images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

                    if config.use_amp:
                        with torch.cuda.amp.autocast():
                            logits, embeddings = model(images)
                            arcface_logits = arcface_loss(embeddings, labels)
                    else:
                        logits, embeddings = model(images)
                        arcface_logits = arcface_loss(embeddings, labels)

                    _, predicted = torch.max(arcface_logits.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

            # Calculate validation accuracy
            if val_total > 0:
                epoch_val_acc = 100. * val_correct / val_total
            else:
                epoch_val_acc = 0.0

        # Calculate metrics
        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_acc = 100. * train_correct / train_total
        current_lr = optimizer.param_groups[0]['lr']
        epoch_time = time.time() - epoch_start_time

        # Update history
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_acc'].append(epoch_val_acc)
        history['learning_rate'].append(current_lr)

        # Print epoch results with timing
        print(f"\n⚡ Epoch {epoch+1}/{config.num_epochs} ({epoch_time:.1f}s):")
        print(f"  🔥 Train Loss: {epoch_train_loss:.4f}")
        print(f"  🎯 Train Acc: {epoch_train_acc:.2f}%")
        if has_validation:
            print(f"  ✅ Val Acc: {epoch_val_acc:.2f}%")
        print(f"  📈 Learning Rate: {current_lr:.2e}")

        # Early stopping and best model saving
        metric_for_best = epoch_val_acc if has_validation else epoch_train_acc
        if metric_for_best > best_val_acc:
            best_val_acc = metric_for_best
            no_improve_count = 0
            
            # Save best model
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'arcface_state_dict': arcface_loss.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_acc': best_val_acc,
                'config': config
            }, f'best_{model_name.lower().replace("-", "")}.pth')
            
            metric_name = "Val" if has_validation else "Train"
            print(f"  🏆 NEW BEST! ({metric_name} Acc: {best_val_acc:.2f}%) - Model saved!")
        else:
            no_improve_count += 1
            print(f"  ⏳ No improvement ({no_improve_count}/{patience})")

        # Early stopping
        if no_improve_count >= patience and epoch >= 5:  # Don't stop too early
            print(f"  🛑 Early stopping triggered! Best {metric_name} Acc: {best_val_acc:.2f}%")
            break

        # Periodic checkpoint
        if (epoch + 1) % config.save_every == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'arcface_state_dict': arcface_loss.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_acc': epoch_val_acc,
                'config': config
            }, f'{model_name.lower().replace("-", "")}_epoch_{epoch+1}.pth')

        cleanup_memory()

    print(f"\n🎉 HIGH-SPEED Training completed for {model_name}!")
    metric_name = "validation" if has_validation else "training"
    print(f"   🏆 Best {metric_name} accuracy: {best_val_acc:.2f}%")
    print(f"   ⚡ Training was OPTIMIZED for maximum GPU utilization!")

    return model, history, best_val_acc

print("✅ HIGH-PERFORMANCE training pipeline ready for MAXIMUM SPEED! 🚀")

In [ ]:
# 🔍 Final Pre-Training Verification
print("🔍 Final verification before training...")

# Check all required components
checks = []

# 1. Check datasets
try:
    assert len(train_dataset) > 0, "Training dataset is empty"
    assert len(test_dataset) > 0, "Test dataset is empty"
    checks.append("✅ Datasets have data")
except Exception as e:
    checks.append(f"❌ Dataset check failed: {e}")

# 2. Check data loaders
try:
    assert len(train_loader) > 0, "Training data loader is empty"
    assert len(test_loader) > 0, "Test data loader is empty"
    checks.append("✅ Data loaders are ready")
except Exception as e:
    checks.append(f"❌ Data loader check failed: {e}")

# 3. Check device and GPU
try:
    print(f"   Device: {device}")
    if torch.cuda.is_available():
        print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    checks.append("✅ Device configured")
except Exception as e:
    checks.append(f"❌ Device check failed: {e}")

# 4. Test data flow
try:
    # Get a sample batch and check shapes
    sample_batch = next(iter(train_loader))
    images, labels = sample_batch

    print(f"   Sample batch - Images: {images.shape}, Labels: {labels.shape}")
    print(f"   Image range: [{images.min():.3f}, {images.max():.3f}]")
    print(f"   Unique labels in batch: {len(torch.unique(labels))}")

    # Test moving to device
    images_gpu = images.to(device)
    labels_gpu = labels.to(device)

    checks.append("✅ Data flow test passed")
except Exception as e:
    checks.append(f"❌ Data flow test failed: {e}")

# 5. Check class count for model initialization
try:
    if hasattr(train_dataset, 'identity_names'):
        num_classes = len(train_dataset.identity_names)
    elif hasattr(train_dataset, 'dataset') and hasattr(train_dataset.dataset, 'identity_names'):
        num_classes = len(train_dataset.dataset.identity_names)
    else:
        # Fallback: count unique labels - sample a few for efficiency
        all_labels = []
        sample_size = min(1000, len(train_dataset))
        for i in range(sample_size):
            _, label = train_dataset[i]
            all_labels.append(label)
        num_classes = len(set(all_labels))

    print(f"   Number of classes for training: {num_classes}")
    assert num_classes > 0, "No classes found"
    checks.append(f"✅ Classes detected: {num_classes}")
except Exception as e:
    checks.append(f"❌ Class count check failed: {e}")

# 6. Test training function
try:
    assert 'train_model' in globals(), "train_model function not defined"
    checks.append("✅ Training function available")
except Exception as e:
    checks.append(f"❌ Training function check failed: {e}")

# 7. Test model architectures with proper eval mode
try:
    # Test SE-ResNet50
    test_se_model = SEResNet50(num_classes=100, embedding_dim=config.embedding_dim)
    test_se_model.eval()  # Set to eval mode to handle BatchNorm properly
    test_input = torch.randn(1, 3, 112, 112)
    
    with torch.no_grad():  # Disable gradients for inference
        test_output = test_se_model(test_input)
    
    assert len(test_output) == 2, "SE-ResNet50 should return logits and embeddings"
    assert test_output[0].shape[0] == 1, f"Expected batch size 1, got {test_output[0].shape[0]}"
    assert test_output[1].shape[1] == config.embedding_dim, f"Expected embedding dim {config.embedding_dim}, got {test_output[1].shape[1]}"
    
    del test_se_model, test_input, test_output
    
    # Test MobileFaceNet
    test_mobile_model = MobileFaceNet(num_classes=100, embedding_dim=config.embedding_dim)
    test_mobile_model.eval()  # Set to eval mode to handle BatchNorm properly
    test_input = torch.randn(1, 3, 112, 112)
    
    with torch.no_grad():  # Disable gradients for inference
        test_output = test_mobile_model(test_input)
    
    assert len(test_output) == 2, "MobileFaceNet should return logits and embeddings"
    assert test_output[0].shape[0] == 1, f"Expected batch size 1, got {test_output[0].shape[0]}"
    assert test_output[1].shape[1] == config.embedding_dim, f"Expected embedding dim {config.embedding_dim}, got {test_output[1].shape[1]}"
    
    del test_mobile_model, test_input, test_output
    
    # Test with actual batch size from data loader
    test_se_model = SEResNet50(num_classes=num_classes, embedding_dim=config.embedding_dim)
    test_se_model.train()  # Set to train mode for batch processing
    test_batch_images, test_batch_labels = next(iter(train_loader))
    
    with torch.no_grad():
        test_batch_output = test_se_model(test_batch_images)
    
    assert len(test_batch_output) == 2, "SE-ResNet50 should return logits and embeddings for batch"
    assert test_batch_output[0].shape[0] == test_batch_images.shape[0], "Batch size mismatch"
    
    del test_se_model, test_batch_images, test_batch_labels, test_batch_output
    
    checks.append("✅ Model architectures work correctly (single sample & batch)")
except Exception as e:
    checks.append(f"❌ Model architecture check failed: {e}")
    import traceback
    print(f"   Detailed error: {traceback.format_exc()}")

# 8. Test ArcFace loss
try:
    test_arcface = ArcFaceLoss(embedding_dim=config.embedding_dim, num_classes=100)
    test_embeddings = torch.randn(16, config.embedding_dim)
    test_labels = torch.randint(0, 100, (16,))
    
    test_arcface_output = test_arcface(test_embeddings, test_labels)
    assert test_arcface_output.shape == (16, 100), f"Expected ArcFace output shape (16, 100), got {test_arcface_output.shape}"
    
    del test_arcface, test_embeddings, test_labels, test_arcface_output
    checks.append("✅ ArcFace loss function works correctly")
except Exception as e:
    checks.append(f"❌ ArcFace loss check failed: {e}")

# Print all check results
print("\n📋 Pre-training Check Results:")
for check in checks:
    print(f"   {check}")

# Final status
all_passed = all("✅" in check for check in checks)
if all_passed:
    print("\n🎉 ALL CHECKS PASSED! Ready to start training! 🚀")
    print("   You can now run the training cells safely.")
    print(f"   📊 Ready to train on {num_classes} classes with {len(train_dataset)} training samples")
else:
    print("\n⚠️ Some checks failed. Please fix the issues above before training.")
    failed_checks = [check for check in checks if "❌" in check]
    print("   Failed checks:")
    for check in failed_checks:
        print(f"     {check}")

cleanup_memory()
print("✅ Verification complete!")

In [6]:
# 🚀 Train SE-ResNet-50
print("🏗️ Initializing SE-ResNet-50...")

# Get number of classes (handle both direct dataset and train/val split)
if hasattr(train_dataset, 'identity_names'):
    num_classes = len(train_dataset.identity_names)
elif hasattr(train_dataset, 'dataset') and hasattr(train_dataset.dataset, 'identity_names'):
    num_classes = len(train_dataset.dataset.identity_names)
else:
    # Fallback: count unique labels in the dataset
    print("   Counting unique labels...")
    all_labels = set()
    for i in range(min(1000, len(train_dataset))):  # Sample first 1000 to avoid long wait
        _, label = train_dataset[i]
        all_labels.add(label)
    num_classes = len(all_labels)

print(f"Number of classes: {num_classes}")

se_resnet50 = SEResNet50(num_classes=num_classes, embedding_dim=config.embedding_dim)

# Train the model
se_resnet50_trained, se_resnet50_history, se_resnet50_best_acc = train_model(
    se_resnet50, "SE-ResNet50", num_classes
)

cleanup_memory()

🏗️ Initializing SE-ResNet-50...
Number of classes: 5547


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 154MB/s]


NameError: name 'train_model' is not defined

In [ ]:
# 🚀 Train MobileFaceNet
print("🏗️ Initializing MobileFaceNet...")

# Use the same num_classes determined from SE-ResNet50 training
print(f"Number of classes: {num_classes}")

mobilefacenet = MobileFaceNet(num_classes=num_classes, embedding_dim=config.embedding_dim)

# Train the model
mobilefacenet_trained, mobilefacenet_history, mobilefacenet_best_acc = train_model(
    mobilefacenet, "MobileFaceNet", num_classes
)

cleanup_memory()

## 5. 🚀 Model Training

In [ ]:
# 🚀 Train SE-ResNet-50
print("🏗️ Initializing SE-ResNet-50...")

# Get number of classes (handle both direct dataset and train/val split)
if hasattr(train_dataset, 'identity_names'):
    num_classes = len(train_dataset.identity_names)
elif hasattr(train_dataset, 'dataset') and hasattr(train_dataset.dataset, 'identity_names'):
    num_classes = len(train_dataset.dataset.identity_names)
else:
    # Fallback: count unique labels in the dataset
    print("   Counting unique labels...")
    all_labels = set()
    for i in range(min(1000, len(train_dataset))):  # Sample first 1000 to avoid long wait
        _, label = train_dataset[i]
        all_labels.add(label)
    num_classes = len(all_labels)

print(f"Number of classes: {num_classes}")

se_resnet50 = SEResNet50(num_classes=num_classes, embedding_dim=config.embedding_dim)

# Train the model
se_resnet50_trained, se_resnet50_history, se_resnet50_best_acc = train_model(
    se_resnet50, "SE-ResNet50", num_classes
)

cleanup_memory()

In [ ]:
# 🚀 Train MobileFaceNet
print("🏗️ Initializing MobileFaceNet...")

mobilefacenet = MobileFaceNet(num_classes=num_classes, embedding_dim=config.embedding_dim)

# Train the model
mobilefacenet_trained, mobilefacenet_history, mobilefacenet_best_acc = train_model(
    mobilefacenet, "MobileFaceNet", num_classes
)

cleanup_memory()

## 6. 📊 Training Results Analysis

In [ ]:
# 📊 Plot training results
def plot_training_results(histories, model_names):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Training loss
    for history, name in zip(histories, model_names):
        axes[0, 0].plot(history['train_loss'], label=f'{name} Train Loss')
    axes[0, 0].set_title('Training Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    # Training accuracy
    for history, name in zip(histories, model_names):
        axes[0, 1].plot(history['train_acc'], label=f'{name} Train Acc')
    axes[0, 1].set_title('Training Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    # Validation accuracy
    for history, name in zip(histories, model_names):
        axes[1, 0].plot(history['val_acc'], label=f'{name} Val Acc')
    axes[1, 0].set_title('Validation Accuracy')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy (%)')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    # Learning rate
    for history, name in zip(histories, model_names):
        axes[1, 1].plot(history['learning_rate'], label=f'{name} LR')
    axes[1, 1].set_title('Learning Rate')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_yscale('log')
    axes[1, 1].legend()
    axes[1, 1].grid(True)

    plt.tight_layout()
    plt.show()

# Plot results
if 'se_resnet50_history' in locals() and 'mobilefacenet_history' in locals():
    plot_training_results(
        [se_resnet50_history, mobilefacenet_history],
        ['SE-ResNet50', 'MobileFaceNet']
    )

    # Print summary
    print("\n🏆 TRAINING SUMMARY:")
    print(f"   SE-ResNet50 Best Val Acc: {se_resnet50_best_acc:.2f}%")
    print(f"   MobileFaceNet Best Val Acc: {mobilefacenet_best_acc:.2f}%")

    better_model = "SE-ResNet50" if se_resnet50_best_acc > mobilefacenet_best_acc else "MobileFaceNet"
    print(f"   🥇 Best performing model: {better_model}")
else:
    print("⚠️ Training histories not available yet")

## 7. 🎯 Ensemble Model Creation

In [ ]:
# 🎯 Ensemble Model
class FaceRecognitionEnsemble(nn.Module):
    def __init__(self, models, weights=None):
        super().__init__()
        self.models = nn.ModuleList(models)
        self.weights = weights if weights else [1.0/len(models)] * len(models)

    def forward(self, x, return_individual=False):
        embeddings = []
        logits = []

        for model in self.models:
            model_logits, model_embedding = model(x)
            logits.append(model_logits)
            embeddings.append(model_embedding)

        if return_individual:
            return logits, embeddings

        # Weighted ensemble of embeddings
        weighted_embedding = sum(w * emb for w, emb in zip(self.weights, embeddings))
        weighted_embedding = F.normalize(weighted_embedding, p=2, dim=1)

        # Weighted ensemble of logits
        weighted_logits = sum(w * logit for w, logit in zip(self.weights, logits))

        return weighted_logits, weighted_embedding

# Create ensemble if both models are trained
if 'se_resnet50_trained' in locals() and 'mobilefacenet_trained' in locals():
    print("🎯 Creating ensemble model...")

    # Load best models
    se_resnet50_trained.load_state_dict(torch.load('best_se-resnet50.pth')['model_state_dict'])
    mobilefacenet_trained.load_state_dict(torch.load('best_mobilefacenet.pth')['model_state_dict'])

    # Set weights based on validation performance
    total_acc = se_resnet50_best_acc + mobilefacenet_best_acc
    se_weight = se_resnet50_best_acc / total_acc
    mobile_weight = mobilefacenet_best_acc / total_acc

    ensemble_model = FaceRecognitionEnsemble(
        models=[se_resnet50_trained, mobilefacenet_trained],
        weights=[se_weight, mobile_weight]
    ).to(device)

    print(f"✅ Ensemble created with weights:")
    print(f"   SE-ResNet50: {se_weight:.3f}")
    print(f"   MobileFaceNet: {mobile_weight:.3f}")

    # Save ensemble
    torch.save({
        'ensemble_state_dict': ensemble_model.state_dict(),
        'weights': [se_weight, mobile_weight],
        'se_resnet50_acc': se_resnet50_best_acc,
        'mobilefacenet_acc': mobilefacenet_best_acc,
        'config': config
    }, 'face_recognition_ensemble.pth')

    print("💾 Ensemble model saved!")
else:
    print("⚠️ Cannot create ensemble - models not trained yet")

## 8. 🔍 Model Evaluation & Testing

In [ ]:
# 🔍 Comprehensive Evaluation
def evaluate_model(model, data_loader, model_name):
    model.eval()
    all_embeddings = []
    all_labels = []
    correct = 0
    total = 0

    print(f"\n🔍 Evaluating {model_name}...")

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluation"):
            images, labels = images.to(device), labels.to(device)

            if hasattr(model, 'models'):  # Ensemble model
                logits, embeddings = model(images)
            else:  # Single model
                logits, embeddings = model(images)

            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Store embeddings and labels for similarity analysis
            all_embeddings.append(embeddings.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100. * correct / total
    all_embeddings = np.vstack(all_embeddings)

    print(f"📊 {model_name} Results:")
    print(f"   Accuracy: {accuracy:.2f}%")
    print(f"   Embedding shape: {all_embeddings.shape}")

    return accuracy, all_embeddings, all_labels

# Face verification evaluation
def face_verification_evaluation(embeddings, labels, num_pairs=1000):
    """Evaluate face verification performance"""
    print("\n🎭 Face Verification Evaluation...")

    # Generate positive and negative pairs
    positive_pairs = []
    negative_pairs = []

    unique_labels = list(set(labels))

    # Generate positive pairs (same identity)
    for label in unique_labels:
        indices = [i for i, l in enumerate(labels) if l == label]
        if len(indices) >= 2:
            for i in range(len(indices)):
                for j in range(i+1, len(indices)):
                    positive_pairs.append((indices[i], indices[j], 1))
                    if len(positive_pairs) >= num_pairs // 2:
                        break
                if len(positive_pairs) >= num_pairs // 2:
                    break

    # Generate negative pairs (different identities)
    import random
    while len(negative_pairs) < num_pairs // 2:
        i, j = random.sample(range(len(labels)), 2)
        if labels[i] != labels[j]:
            negative_pairs.append((i, j, 0))

    all_pairs = positive_pairs + negative_pairs
    random.shuffle(all_pairs)

    # Calculate similarities and evaluate
    similarities = []
    true_labels = []

    for i, j, label in all_pairs:
        # Cosine similarity
        sim = np.dot(embeddings[i], embeddings[j]) / (
            np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
        )
        similarities.append(sim)
        true_labels.append(label)

    similarities = np.array(similarities)
    true_labels = np.array(true_labels)

    # Find best threshold
    best_threshold = 0.5
    best_accuracy = 0

    for threshold in np.arange(0.1, 1.0, 0.05):
        predictions = (similarities > threshold).astype(int)
        accuracy = (predictions == true_labels).mean()
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold

    print(f"   📊 Verification Results:")
    print(f"      Best threshold: {best_threshold:.3f}")
    print(f"      Best accuracy: {best_accuracy:.3f}")
    print(f"      Similarity range: [{similarities.min():.3f}, {similarities.max():.3f}]")

    return best_threshold, best_accuracy, similarities

# Evaluate all models if available
evaluation_results = {}

if 'se_resnet50_trained' in locals():
    se_acc, se_embeddings, se_labels = evaluate_model(se_resnet50_trained, test_loader, "SE-ResNet50")
    se_threshold, se_ver_acc, se_similarities = face_verification_evaluation(se_embeddings, se_labels)
    evaluation_results['SE-ResNet50'] = {
        'accuracy': se_acc,
        'verification_accuracy': se_ver_acc,
        'threshold': se_threshold
    }

if 'mobilefacenet_trained' in locals():
    mobile_acc, mobile_embeddings, mobile_labels = evaluate_model(mobilefacenet_trained, test_loader, "MobileFaceNet")
    mobile_threshold, mobile_ver_acc, mobile_similarities = face_verification_evaluation(mobile_embeddings, mobile_labels)
    evaluation_results['MobileFaceNet'] = {
        'accuracy': mobile_acc,
        'verification_accuracy': mobile_ver_acc,
        'threshold': mobile_threshold
    }

if 'ensemble_model' in locals():
    ensemble_acc, ensemble_embeddings, ensemble_labels = evaluate_model(ensemble_model, test_loader, "Ensemble")
    ensemble_threshold, ensemble_ver_acc, ensemble_similarities = face_verification_evaluation(ensemble_embeddings, ensemble_labels)
    evaluation_results['Ensemble'] = {
        'accuracy': ensemble_acc,
        'verification_accuracy': ensemble_ver_acc,
        'threshold': ensemble_threshold
    }

# Print final results
print("\n🏆 FINAL EVALUATION RESULTS:")
print("=" * 50)
for model_name, results in evaluation_results.items():
    print(f"{model_name}:")
    print(f"   Classification Accuracy: {results['accuracy']:.2f}%")
    print(f"   Verification Accuracy: {results['verification_accuracy']:.3f}")
    print(f"   Optimal Threshold: {results['threshold']:.3f}")
    print()

cleanup_memory()

## 9. 💾 Model Export & Deployment

In [ ]:
# 💾 Export models for deployment
print("💾 Exporting models for deployment...")

# Export individual models
if 'se_resnet50_trained' in locals():
    se_resnet50_trained.eval()
    dummy_input = torch.randn(1, 3, 112, 112).to(device)

    # TorchScript export
    se_resnet50_script = torch.jit.trace(se_resnet50_trained, dummy_input)
    se_resnet50_script.save('se_resnet50_deployment.pt')
    print("✅ SE-ResNet50 exported to se_resnet50_deployment.pt")

if 'mobilefacenet_trained' in locals():
    mobilefacenet_trained.eval()

    # TorchScript export
    mobilefacenet_script = torch.jit.trace(mobilefacenet_trained, dummy_input)
    mobilefacenet_script.save('mobilefacenet_deployment.pt')
    print("✅ MobileFaceNet exported to mobilefacenet_deployment.pt")

if 'ensemble_model' in locals():
    ensemble_model.eval()

    # TorchScript export
    ensemble_script = torch.jit.trace(ensemble_model, dummy_input)
    ensemble_script.save('ensemble_deployment.pt')
    print("✅ Ensemble model exported to ensemble_deployment.pt")

# Create deployment configuration
deployment_config = {
    'image_size': 112,
    'embedding_dim': config.embedding_dim,
    'normalization': {
        'mean': [0.485, 0.456, 0.406],
        'std': [0.229, 0.224, 0.225]
    },
    'evaluation_results': evaluation_results,
    'optimal_thresholds': {name: results['threshold'] for name, results in evaluation_results.items()}
}

import json
with open('deployment_config.json', 'w') as f:
    json.dump(deployment_config, f, indent=2)

print("✅ Deployment configuration saved to deployment_config.json")

# Summary
print("\n🎉 TRAINING AND EXPORT COMPLETE!")
print("📁 Generated files:")
print("   - best_se-resnet50.pth (training checkpoint)")
print("   - best_mobilefacenet.pth (training checkpoint)")
print("   - face_recognition_ensemble.pth (ensemble checkpoint)")
print("   - se_resnet50_deployment.pt (TorchScript)")
print("   - mobilefacenet_deployment.pt (TorchScript)")
print("   - ensemble_deployment.pt (TorchScript)")
print("   - deployment_config.json (configuration)")

print("\n🚀 Ready for production deployment!")